In [ ]:
!git clone https://github.com/rhisehl/Customer-Churn-Model

fatal: destination path 'Customer-Churn-Model' already exists and is not an empty directory.


In [ ]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go 
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import * 
from sklearn.svm import SVC

In [ ]:
# Set up S3 import
!pip install boto3
import boto3

BUCKET_NAME = 'ecommerce-machine-learning-resources-5.29.23'

from config import aws_access_key_id, aws_secret_access_key, region

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Authentication credentials for S3
s3 = boto3.resource('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
KEY = 'cleaned_data.csv'
# Import file
s3.Bucket(BUCKET_NAME).download_file(KEY,'cleaned_data.csv')

In [ ]:
# Read CSV into DataFrame
df = pd.read_csv('cleaned_data.csv',index_col=0)
df.set_index("CustomerID",inplace=True)
df

,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,AppHours,RegisteredDevices,PreferredCategory,SatisfactionScore,MaritalStatus,...,Complaint,OrderIncreasePercent,CouponsUsed,OrderCount,DaysSinceOrder,AverageCashback,Age,Race,HouseholdIncome,PersonalIncome
CustomerID,,,,,,,,,,,,,,,,,,,,,
50001,1,4.0,Mobile Phone,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,...,1,11.0,1.0,1.0,5.0,160,80,White,53704.40,53704.40
50004,1,0.0,Mobile Phone,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,...,0,23.0,0.0,1.0,3.0,134,20,Other,99863.00,6798.03
50006,1,0.0,Computer,Debit Card,Female,3.0,5,Mobile Phone,5,Single,...,1,22.0,4.0,6.0,7.0,139,60,White,50985.19,50985.19
50012,1,11.0,Mobile Phone,Debit Card,Male,3.0,4,Fashion,3,Single,...,1,13.0,0.0,1.0,0.0,154,29,Black,53024.60,16995.06
50013,1,0.0,Mobile Phone,Cash on Delivery,Male,2.0,3,Mobile,3,Single,...,1,13.0,2.0,2.0,2.0,134,39,White,18184.72,18184.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55625,0,1.0,Mobile Phone,Unified Payment Interface,Female,2.0,5,Mobile Phone,3,Single,...,0,19.0,2.0,2.0,1.0,155,24,White,191194.47,63731.49
55626,0,10.0,Computer,Credit Card,Male,3.0,2,Laptop & Accessory,1,Married,...,0,18.0,1.0,2.0,4.0,151,36,White,103160.04,101970.38
55628,0,1.0,Mobile Phone,Debit Card,Male,3.0,2,Laptop & Accessory,4,Married,...,1,21.0,1.0,2.0,4.0,186,63,White,95172.36,64581.24


In [ ]:
clean_df = df

# convert categorical data to numeric with `pd.get_dummies`
df2 = clean_df.copy()
df = pd.get_dummies(df2)
df

,Churn,Tenure,AppHours,RegisteredDevices,SatisfactionScore,NumberAddresses,Complaint,OrderIncreasePercent,CouponsUsed,OrderCount,...,MaritalStatus_Married,MaritalStatus_Single,Race_American Indian or Alaska Native,Race_Black,Race_Chinese,Race_Japanese,Race_Other,Race_Other Asian or Pacific Islander,Race_Two major races,Race_White
CustomerID,,,,,,,,,,,,,,,,,,,,,
50001,1,4.0,3.0,3,2,9,1,11.0,1.0,1.0,...,0,1,0,0,0,0,0,0,0,1
50004,1,0.0,2.0,4,5,8,0,23.0,0.0,1.0,...,0,1,0,0,0,0,1,0,0,0
50006,1,0.0,3.0,5,5,2,1,22.0,4.0,6.0,...,0,1,0,0,0,0,0,0,0,1
50012,1,11.0,3.0,4,3,10,1,13.0,0.0,1.0,...,0,1,0,1,0,0,0,0,0,0
50013,1,0.0,2.0,3,3,2,1,13.0,2.0,2.0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55625,0,1.0,2.0,5,3,2,0,19.0,2.0,2.0,...,0,1,0,0,0,0,0,0,0,1
55626,0,10.0,3.0,2,1,6,0,18.0,1.0,2.0,...,1,0,0,0,0,0,0,0,0,1
55628,0,1.0,3.0,2,4,3,1,21.0,1.0,2.0,...,1,0,0,0,0,0,0,0,0,1


In [ ]:
# Reset dataset to begin with original data again
def start_dataset():

  # Separate the data into labels and features
    # the y variable, the labels (0 = Active Customer, 1 = Inactive Customer)
  y = df['Churn']
    # the X variable, the features
  X = df.drop(columns='Churn')

  # Split into testing and training sets using train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
  X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, random_state=1)

  # recombobulate datasets
  train = X_train.join(y_train)
  valid = X_valid.join(y_valid)
  test = X_test.join(y_test)

  return X_train, X_valid, X_test, y_train, y_valid, y_test, train, valid, test, X, y

In [ ]:
# Scale (normalize) the dataset to avoid skewing caused by numbers of various sizes

def scale_dataset(train, oversample=False, scaler=RobustScaler(quantile_range=(25, 75))):
    y = df['Churn'].values
    X = df.drop(columns='Churn').values

    # Instantiate a StandardScaler instance
    scaler = scaler

    # fit the data and transform
    X = scaler.fit_transform(X)
    
    # if we decide to use the RandomOverSampler module due to a large variance in record numbers
    ## to take more of a less class and increase the size of the dataset
    if oversample:
        ros = RandomOverSampler(sampling_strategy='minority')
        X, y = ros.fit_resample(X, y)
        
    # create one huge 2d numpy array (hstack, horizontal stack side by side)
    train = np.hstack((X, np.reshape(y, (-1,1))))
    
    # return fit and transformed data
    return train, X, y

In [ ]:
# Generate confusion matrix and classification report information for print report and csv file processing
def confusion_classification(y_test, y_pred):

    # generate a confusion matrix for the model
    cm_df_init = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(
            cm_df_init, index=['Actual Active Customer', 'Actual Inactive Customer'],
            columns=['Predicted Active Customer', 'Predicted Inactive Customer']
    )
    
    # generate classification report dataframe details
    classification_df = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_pred, target_names=churn, output_dict=True)).transpose()

    return(cm_df, classification_df)   

In [ ]:
# Begin processing
X_train, X_valid, X_test, y_train, y_valid, y_test, train, valid, test, X, y = start_dataset()

In [ ]:
# Verify all rows accounted for
df.shape[0], train.shape[0] + valid.shape[0] + test.shape[0]

(4019, 4019)

In [ ]:
# train.info(), test.info(), valid.info()

## Support Vector Machine (SVM) Model with top score

In [ ]:
# Create variables and re-run before optimizing to confirm origimal model set up
churn =['Active', 'Inactive']
oversamples = ['True']
scalers=[RobustScaler(quantile_range=(25, 75))]

# Reset dataset
X_train, X_valid, X_test, y_train, y_valid, y_test, train, valid, test, X, y = start_dataset()
classification_df = []
neighbor_num = 0

run_model = 'Support Vector Machines (SVM)'
# Fit the model using the original then the resampled training data,
for oversample in oversamples:
  for scaler in scalers:
    # scale (normalize) the train and test datasets     
    train, X_train, y_train = scale_dataset(train, oversample=oversample)
    valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
    test, X_test, y_test = scale_dataset(test, oversample=False)
    
    # Fit the model using the original then the resampled training data
    svm_model = SVC()
    svm_model = svm_model.fit(X_train, y_train)
    
    # make a prediction using the testing data
    y_pred = svm_model.predict(X_test)
    
    # generate a confusion matrix and classification data for the csv for the model
    cm_df, classification_df = confusion_classification(y_test, y_pred)

    # print the classification reports
    print(('*' * 50)+'\n')  
    print(f'{run_model} Model Reports')
    print(f'Parameters: oversample={oversample} and scaler={scaler}\n')
    # print('Confusion Matrix')
    # display(cm_df)
    print(f'Classification Report')
    print(classification_report(y_test, y_pred, target_names=churn)) 

**************************************************

Support Vector Machines (SVM) Model Reports
Parameters: oversample=True and scaler=RobustScaler(quantile_range=(25, 75))

Classification Report
              precision    recall  f1-score   support

      Active       0.98      0.89      0.94      3304
    Inactive       0.65      0.94      0.77       715

    accuracy                           0.90      4019
   macro avg       0.82      0.92      0.85      4019
weighted avg       0.93      0.90      0.91      4019



In [ ]:
# Test one major parameter which returns the corresponding SVC model
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

In [ ]:
# Let's determine which kernel performs best
for i in range(4):
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)# Make prediction
    y_pred = svclassifier.predict(X_test)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       1.00      0.16      0.27      3304
           1       0.20      1.00      0.34       715

    accuracy                           0.31      4019
   macro avg       0.60      0.58      0.31      4019
weighted avg       0.86      0.31      0.28      4019

Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.97      0.85      0.91      3304
           1       0.56      0.87      0.68       715

    accuracy                           0.85      4019
   macro avg       0.76      0.86      0.79      4019
weighted avg       0.89      0.85      0.87      4019

Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.92      0.70      0.80      3304
           1       0.34      0.71      0.46       715

    accuracy                           0.70      4019
   macro avg       0.63      0.70      0.63     

# SVM Hyperparameter Tuning using GridSearchCV
### 3 Major parameters for tuning:
## Kernels - takes low dimensional input and make higher
## C(Regularisation) - penalty parm, how much error is bearable
## Gamma - Most Influence: High value = near points; Low value = far away points

In [ ]:
# Tune Hyperparameters with GridSearchCV
parms = {
'C' : [0.1, 1, 10, 100],
'gamma' : [1, 0.1, 0.01, 0.001],
'kernel' : ['rbf', 'sigmoid', 'linear']
}       

# Other parameters available:
# break_ties : 
# cache_size : 
# class_weight : 
# coef0 : 
# decision_function_shape : 
# degree : 
# max_iter : 
# probability : 
# random_state :
# shrinking :
# tol : 

grid = GridSearchCV(SVC(), parms, verbose=2, refit=True)
bestF = grid.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   2.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   2.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   2.2s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   1.9s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   1.8s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   2.2s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   4.2s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   3.6s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   1.3s
[CV] END ......................C=0.1, gamma=1, 

In [ ]:
# Optimization best fit!!
print(bestF.best_params_)
print(bestF.best_score_)

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}
0.9950073352489346


In [18]:
# Make a prediction using the testing data
y_pred = grid.predict(X_test)

# generate a confusion matrix and classification data for the csv for the model
cm_df, classification_df = confusion_classification(y_test, y_pred)

# print the classification reports
print(('*' * 50)+'\n')  
print(f'{run_model} Model Post Optimization Report')
print(f"Parameters: 'C': 1, 'gamma': 1, 'kernel': 'rbf' \n")
# print('Confusion Matrix')
# display(cm_df)
# print('\n')  
print(f'Classification Report')
print(classification_report(y_test, y_pred, target_names=churn)) 

**************************************************

Support Vector Machines (SVM) Model Post Optimization Report
Parameters: 'C': 1, 'gamma': 1, 'kernel': 'rbf' 

Classification Report
              precision    recall  f1-score   support

      Active       1.00      1.00      1.00      3304
    Inactive       1.00      1.00      1.00       715

    accuracy                           1.00      4019
   macro avg       1.00      1.00      1.00      4019
weighted avg       1.00      1.00      1.00      4019

